## Taller 9
### Ahora con otra red y datos

Importación y transformación de los datos

In [28]:
import pandas as pd

columnas = ["age","sex","cp","trestbps","chol","fbs","restecg","thalach","exang","oldpeak","slope","ca","thal","num"]
datos_iniciales = pd.read_csv('https://raw.githubusercontent.com/karen0c/Proyecto_1_ACTD/main/processed.cleveland.data',header=None, names=columnas, na_values="?")

datos_iniciales = datos_iniciales.dropna().reset_index(drop=True) #elimina filas con valores faltantes
#print(datos_iniciales.head())
#print(datos_iniciales.tail())

# crear unos nuevos datos donde guardaremos la información con los datos discretizados
datos = datos_iniciales

for i in range(0,297):
  if datos_iniciales.loc[i, 'age'] <= 40:
    datos.loc[i, 'age'] = 1
  elif datos_iniciales.loc[i, 'age'] > 40 and datos_iniciales.loc[i, 'age'] <= 50:
    datos.loc[i, 'age'] = 2
  elif datos_iniciales.loc[i, 'age'] > 50 and datos_iniciales.loc[i, 'age'] <= 60:
    datos.loc[i, 'age'] = 3
  else:
    datos.loc[i, 'age'] = 4

for i in range(0,297):
  if datos_iniciales.loc[i, "trestbps"] <= 120:
    datos.loc[i, "trestbps"] = 1
  elif datos_iniciales.loc[i, "trestbps"] > 120 and datos_iniciales.loc[i, "trestbps"] <= 139:
    datos.loc[i, "trestbps"] = 2
  elif datos_iniciales.loc[i, "trestbps"] >= 140 and datos_iniciales.loc[i, "trestbps"] <= 159:
    datos.loc[i, "trestbps"] = 3
  elif datos_iniciales.loc[i, "trestbps"] >= 160 and datos_iniciales.loc[i, "trestbps"] <= 179:
    datos.loc[i, "trestbps"] = 4
  else:
    datos.loc[i, "trestbps"] = 5
    
for i in range(0,297):
  if datos_iniciales.loc[i, "chol"] <= 200:
    datos.loc[i, "chol"] = 1
  elif datos_iniciales.loc[i, "chol"] > 200 and datos_iniciales.loc[i, "chol"] < 240:
    datos.loc[i, "chol"] = 2
  else:
    datos.loc[i, "chol"] = 3
    
for i in range(0,297):
  if datos_iniciales.loc[i, "thalach"] <= 120:
    datos.loc[i, "thalach"] = 1
  elif datos_iniciales.loc[i, "thalach"] > 120 and datos_iniciales.loc[i, "thalach"] <= 140:
    datos.loc[i, "thalach"] = 2
  elif datos_iniciales.loc[i, "thalach"] > 140 and datos_iniciales.loc[i, "thalach"] < 160:
    datos.loc[i, "thalach"] = 3
  else:
    datos.loc[i, "thalach"] = 4
    
for i in range(0,297):
  if datos_iniciales.loc[i, "oldpeak"] <= 1:
    datos.loc[i, "oldpeak"] = 1
  elif datos_iniciales.loc[i, "oldpeak"] > 1 and datos_iniciales.loc[i, "oldpeak"] <= 2:
    datos.loc[i, "oldpeak"] = 2
  else:
    datos.loc[i, "oldpeak"] = 3

#USAR SOLO EL 80% DE LOS DATOS, PARA QUE SIRVAN COMO ENTRENAMIENTO, LO DEMÁS SE USARÁ COMO TEST
sep = int(0.8*len(datos))
datos_train = datos[:sep]
print(datos_train.head(10))


   age  sex   cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
0  4.0  1.0  1.0       3.0   2.0  1.0      2.0      3.0    0.0      3.0   
1  4.0  1.0  4.0       4.0   3.0  0.0      2.0      1.0    1.0      2.0   
2  4.0  1.0  4.0       1.0   2.0  0.0      2.0      2.0    1.0      3.0   
3  1.0  1.0  3.0       2.0   3.0  0.0      0.0      4.0    0.0      3.0   
4  2.0  0.0  2.0       2.0   2.0  0.0      2.0      4.0    0.0      2.0   
5  3.0  1.0  2.0       1.0   2.0  0.0      0.0      4.0    0.0      1.0   
6  4.0  0.0  4.0       3.0   3.0  0.0      2.0      4.0    0.0      3.0   
7  3.0  0.0  4.0       1.0   3.0  0.0      0.0      4.0    1.0      1.0   
8  4.0  1.0  4.0       2.0   3.0  0.0      2.0      3.0    0.0      2.0   
9  3.0  1.0  4.0       3.0   2.0  1.0      2.0      3.0    1.0      3.0   

   slope   ca  thal  num  
0    3.0  0.0   6.0    0  
1    2.0  3.0   3.0    2  
2    2.0  2.0   7.0    1  
3    3.0  0.0   3.0    0  
4    1.0  0.0   3.0    0  
5    1.0  0.

In [29]:
print(sep)

237


In [30]:
datos_prueba = datos[sep+1:]
print(datos_prueba.head(10))

     age  sex   cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
238  2.0  0.0  2.0       2.0   3.0  0.0      0.0      4.0    0.0      1.0   
239  2.0  0.0  4.0       2.0   3.0  0.0      0.0      4.0    0.0      1.0   
240  4.0  1.0  1.0       2.0   2.0  0.0      0.0      3.0    0.0      3.0   
241  3.0  0.0  3.0       1.0   1.0  1.0      0.0      1.0    0.0      1.0   
242  4.0  1.0  4.0       1.0   2.0  0.0      0.0      1.0    0.0      1.0   
243  3.0  1.0  4.0       1.0   2.0  0.0      0.0      3.0    0.0      1.0   
244  2.0  1.0  4.0       1.0   3.0  0.0      2.0      1.0    1.0      1.0   
245  3.0  1.0  4.0       2.0   2.0  0.0      0.0      4.0    0.0      1.0   
246  4.0  1.0  2.0       2.0   2.0  1.0      2.0      2.0    0.0      1.0   
247  3.0  1.0  4.0       1.0   2.0  0.0      0.0      2.0    1.0      2.0   

     slope   ca  thal  num  
238    1.0  0.0   3.0    0  
239    1.0  0.0   3.0    0  
240    2.0  2.0   3.0    2  
241    1.0  0.0   3.0    0  
242    

Transforme las variables de la siguiente manera

In [5]:
TIERS_NUM = 3

def boundary_str(start, end, tier):
    return f'{tier}: {start:+0,.2f} to {end:+0,.2f}'

def relabel_value(v, boundaries):
    if v >= boundaries[0][0] and v <= boundaries[0][1]:
        return boundary_str(boundaries[0][0], boundaries[0][1], tier='A')
    elif v >= boundaries[1][0] and v <= boundaries[1][1]: 
        return boundary_str(boundaries[1][0], boundaries[1][1], tier='B')
    elif v >= boundaries[2][0] and v <= boundaries[2][1]:
        return boundary_str(boundaries[2][0], boundaries[2][1], tier='C')
    else:
        return np.nan

def relabel(values, boundaries):
    result = []
    for v in values:
        result.append(relabel_value(v, boundaries))
    return result
        
def get_boundaries(tiers):
    prev_tier = tiers[0]
    boundaries = [(prev_tier[0], prev_tier[prev_tier.shape[0] - 1])]
    for index, tier in enumerate(tiers):
        if index is not 0:
            boundaries.append((prev_tier[prev_tier.shape[0] - 1], tier[tier.shape[0] - 1]))
            prev_tier = tier
    return boundaries
    
new_columns = {}
for i, content in enumerate(datos_train.items()):  
    (label, series) = content
    values = np.sort(np.array([x for x in series.tolist() if not np.isnan(x)] , dtype=float))
    if values.shape[0] < TIERS_NUM:
        print(f'Error: there are not enough data for label {label}')
        break
    boundaries = get_boundaries(tiers=np.array_split(values, TIERS_NUM)) 
    new_columns[label] = relabel(series.tolist(), boundaries)
    
df = DataFrame(data=new_columns)
#df.columns = nodes
#df.index = range(years["min"], years["max"] + 1)
print(df.head(10))

                 age                sex                 cp           trestbps  \
0  C: +3.00 to +4.00  A: +0.00 to +1.00  A: +1.00 to +3.00  B: +2.00 to +3.00   
1  C: +3.00 to +4.00  A: +0.00 to +1.00  B: +3.00 to +4.00  C: +3.00 to +5.00   
2  C: +3.00 to +4.00  A: +0.00 to +1.00  B: +3.00 to +4.00  A: +1.00 to +2.00   
3  A: +1.00 to +3.00  A: +0.00 to +1.00  A: +1.00 to +3.00  A: +1.00 to +2.00   
4  A: +1.00 to +3.00  A: +0.00 to +1.00  A: +1.00 to +3.00  A: +1.00 to +2.00   
5  A: +1.00 to +3.00  A: +0.00 to +1.00  A: +1.00 to +3.00  A: +1.00 to +2.00   
6  C: +3.00 to +4.00  A: +0.00 to +1.00  B: +3.00 to +4.00  B: +2.00 to +3.00   
7  A: +1.00 to +3.00  A: +0.00 to +1.00  B: +3.00 to +4.00  A: +1.00 to +2.00   
8  C: +3.00 to +4.00  A: +0.00 to +1.00  B: +3.00 to +4.00  A: +1.00 to +2.00   
9  A: +1.00 to +3.00  A: +0.00 to +1.00  B: +3.00 to +4.00  B: +2.00 to +3.00   

                chol                fbs            restecg            thalach  \
0  A: +1.00 to +2.00  C: +0

<>:26: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:26: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<ipython-input-5-bf97ec38afae>:26: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if index is not 0:


Estime la estructura del modelo usando el método por restricciones

In [8]:
from pgmpy.estimators import PC
est = PC(data=df)

estimated_model = est.estimate(variant="stable", max_cond_vars=4)
print(estimated_model)
print(estimated_model.nodes())
print(estimated_model.edges())

  0%|          | 0/4 [00:00<?, ?it/s]


['age', 'ca', 'cp', 'num', 'thal', 'exang', 'oldpeak', 'slope']
[('age', 'ca'), ('cp', 'num'), ('thal', 'num'), ('exang', 'cp'), ('slope', 'oldpeak')]


Se convierte el objeto DAG obtenido con el anterior procedimiento a una red bayesiana y se usa el estimador de máxima verosimilitud para estimar los parámetros de la red.

In [9]:
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator

estimated_model = BayesianNetwork(estimated_model)
estimated_model.fit(data=df, estimator = MaximumLikelihoodEstimator) 
for i in estimated_model.nodes():
    print(estimated_model.get_cpds(i))

+------------------------+----------+
| age(A: +1.00 to +3.00) | 0.742616 |
+------------------------+----------+
| age(C: +3.00 to +4.00) | 0.257384 |
+------------------------+----------+
+-----------------------+------------------------+------------------------+
| age                   | age(A: +1.00 to +3.00) | age(C: +3.00 to +4.00) |
+-----------------------+------------------------+------------------------+
| ca(A: +0.00 to +0.00) | 0.6704545454545454     | 0.36065573770491804    |
+-----------------------+------------------------+------------------------+
| ca(B: +0.00 to +1.00) | 0.19886363636363635    | 0.2459016393442623     |
+-----------------------+------------------------+------------------------+
| ca(C: +1.00 to +3.00) | 0.13068181818181818    | 0.39344262295081966    |
+-----------------------+------------------------+------------------------+
+-----------------------+--------------------------+--------------------------+
| exang                 | exang(A: +0.00 to +0

## Estime la estructura del modelo usando el método por puntajes

In [16]:
from pgmpy.estimators import HillClimbSearch
from pgmpy.estimators import K2Score

scoring_method = K2Score(data=df)
esth = HillClimbSearch(data=df)
estimated_modelK2 = esth.estimate(
    scoring_method=scoring_method, max_indegree=4, max_iter=int(1e4)
)
print(estimated_modelK2)
print(estimated_modelK2.nodes())
print(estimated_modelK2.edges())

  0%|          | 0/10000 [00:00<?, ?it/s]


['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'num']
[('age', 'ca'), ('age', 'thalach'), ('age', 'chol'), ('cp', 'exang'), ('cp', 'restecg'), ('trestbps', 'age'), ('restecg', 'chol'), ('thalach', 'restecg'), ('exang', 'thalach'), ('oldpeak', 'restecg'), ('slope', 'oldpeak'), ('slope', 'thalach'), ('ca', 'num'), ('num', 'thal'), ('num', 'cp'), ('num', 'exang'), ('num', 'slope'), ('num', 'oldpeak')]


Se imprime el resultado del puntaje obtenido

In [18]:
print(scoring_method.score(estimated_modelK2))

-2133.9431469792644


### El procedimiento anterior usando el punta BIC

In [15]:
from pgmpy.estimators import BicScore

scoring_method = BicScore(data=df)
esth = HillClimbSearch(data=df)
estimated_modelBic = esth.estimate(
    scoring_method=scoring_method, max_indegree=4, max_iter=int(1e4)
)
print(estimated_modelBic)
print(estimated_modelBic.nodes())
print(estimated_modelBic.edges())

  0%|          | 0/10000 [00:00<?, ?it/s]


['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'num']
[('age', 'chol'), ('cp', 'exang'), ('chol', 'restecg'), ('exang', 'thalach'), ('slope', 'oldpeak'), ('slope', 'thalach'), ('ca', 'age'), ('thal', 'num'), ('num', 'cp'), ('num', 'ca'), ('num', 'slope'), ('num', 'exang')]


Se imprime el resultado del puntaje obtenido

In [19]:
print(scoring_method.score(estimated_modelBic))

-2142.3546863442734


### Predicción

In [50]:
# Importar librerías necesarias
from pgmpy.inference import VariableElimination

In [52]:
print(nueva_base.index)

RangeIndex(start=238, stop=297, step=1)


In [53]:
print(range(len(nueva_base.index)))

range(0, 59)


In [59]:
for i in nueva_base.index:
    print(i)

238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296


In [70]:
# Crear una lista vacía para contener las variables del modelo
variables_modelo = []

# Recorrer cada variable del modelo y agregarla a la lista si está presente en la base de datos original
for nodo in estimated_model.nodes():
    if nodo in datos_prueba.columns:
        variables_modelo.append(datos_prueba[nodo])

# Concatenar todas las variables en una sola base de datos
nueva_base = pd.concat(variables_modelo, axis=1)
print(nueva_base.head())


     age   ca   cp  num  thal  exang  oldpeak  slope
238  2.0  0.0  2.0    0   3.0    0.0      1.0    1.0
239  2.0  0.0  4.0    0   3.0    0.0      1.0    1.0
240  4.0  2.0  1.0    2   3.0    0.0      3.0    2.0
241  3.0  0.0  3.0    0   3.0    0.0      1.0    1.0
242  4.0  0.0  4.0    2   3.0    0.0      1.0    2.0


In [80]:
sex_states = estimated_model.get_cpds("age").state_names["age"]
sex_unique = nueva_base["age"].unique()

print(set(sex_states))
print(set(sex_unique) - set(sex_states))


{'C: +3.00 to +4.00', 'A: +1.00 to +3.00'}
{1.0, 2.0, 3.0, 4.0}


In [74]:
# Crear un objeto de inferencia basado en el modelo entrenado
infer = VariableElimination(estimated_model)

# Calcular las predicciones para cada fila en los datos de prueba
for i in (nueva_base.index):
    evidence = nueva_base.loc[i].to_dict()
    evidence.pop('num', None)  # Elimina la clave 'num' del diccionario si está presente
    prediccion = infer.query(['num'], evidence=evidence)


KeyError: 2.0

In [45]:
datos_prueba.loc[238]

age         2.0
sex         0.0
cp          2.0
trestbps    2.0
chol        3.0
fbs         0.0
restecg     0.0
thalach     4.0
exang       0.0
oldpeak     1.0
slope       1.0
ca          0.0
thal        3.0
num         0.0
Name: 238, dtype: float64